In [23]:
import re
import json
import csv
import requests
from bs4 import BeautifulSoup ,SoupStrainer
import pandas as pd 
from io import StringIO
from pathlib import Path  

#!pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import sys

import unittest

import time


In [24]:
def get_infinity_page_html_data(chromedriver_path="./chromedriver.exe", scroll_number=50, sleep_time=2, url = ""):
    # Create selenium driver
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    for i in range(1,scroll_number):
        print(f"scroll_number: {i}")
        driver.execute_script("window.scrollTo(1,100000)")
        time.sleep(sleep_time)
    html_data = driver.page_source
    driver.close()
    
    soup = BeautifulSoup(html_data, 'html.parser')    
    return soup


In [25]:
def make_table(soup):
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    return df

In [26]:
def data_cleaning(df):
    df = df[df["Open"].str.contains("Dividend") == False]
    df = df[df["Open"].str.contains("Stock") == False]
    df.drop(df.tail(1).index,inplace=True)
    return df

In [27]:
def write_to_file(df, path):
    from pathlib import Path
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filepath) 

In [54]:
base_url = "https://finance.yahoo.com/quote/"
period_url = "/history?period1=1262304000&period2=1651881600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    soup = get_infinity_page_html_data(url = (base_url + index + period_url))
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    
    df = data_cleaning(df)
    
    path = index + ".csv"
    write_to_file(df, path)
    

scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9
scroll_number: 10
scroll_number: 11
scroll_number: 12
scroll_number: 13
scroll_number: 14
scroll_number: 15
scroll_number: 16
scroll_number: 17
scroll_number: 18
scroll_number: 19
scroll_number: 20
scroll_number: 21
scroll_number: 22
scroll_number: 23
scroll_number: 24
scroll_number: 25
scroll_number: 26
scroll_number: 27
scroll_number: 28
scroll_number: 29
scroll_number: 30
scroll_number: 31
scroll_number: 32
scroll_number: 33
scroll_number: 34
scroll_number: 35
scroll_number: 36
scroll_number: 37
scroll_number: 38
scroll_number: 39
scroll_number: 40
scroll_number: 41
scroll_number: 42
scroll_number: 43
scroll_number: 44
scroll_number: 45
scroll_number: 46
scroll_number: 47
scroll_number: 48
scroll_number: 49
scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_n

In [62]:
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    
    df = pd.read_csv( index + '.csv')
    df.drop(df.index[2500:], inplace=True)
    df.set_index('Date')
   
    
    main_df['Date'] = df['Date']
    main_df[index + " " + "Open"] = df['Open']
    main_df[index + " " + "High"] = df['High']
    main_df[index + " " + "Low"] = df['Low']
    main_df[index + " " + "Close"] = df['Close*']
    main_df[index + " " + "Volume"] = df['Volume']
    main_df[index + " " +'Daily change'] = (((df['Close*']).astype(float))-((df['Open']).astype(float)))/((df['Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((df['High'].astype(float)-df['Low'].astype(float))/df['Low'].astype(float)*100)
    


    
    print(df)
    print(df.info())


main_df['Date'] = df['Date']
main_df.set_index('Date')
write_to_file(main_df, ("main_dataframe" + ".csv"))
main_df

      Unnamed: 0          Date    Open    High     Low  Close*  Adj Close**  \
0              0  May 06, 2022  187.36  195.12  179.90  186.75       186.75   
1              1  May 05, 2022  198.67  199.25  185.00  188.44       188.44   
2              2  May 04, 2022  199.23  204.00  187.51  203.34       203.34   
3              3  May 03, 2022  194.00  198.25  191.33  196.02       196.02   
4              4  May 02, 2022  185.41  195.74  183.91  195.33       195.33   
...          ...           ...     ...     ...     ...     ...          ...   
2495        2534  Jun 07, 2012    3.14    3.16    2.97    2.97         2.73   
2496        2535  Jun 06, 2012    3.04    3.11    3.02    3.10         2.84   
2497        2536  Jun 05, 2012    2.92    3.03    2.92    3.02         2.77   
2498        2537  Jun 04, 2012    3.01    3.03    2.91    2.93         2.69   
2499        2538  Jun 01, 2012    3.04    3.07    2.99    2.99         2.75   

        Volume  
0     63299300  
1     62633100  


,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
0,"May 06, 2022",187.36,195.12,179.90,186.75,63299300,-0.325576,8.460256,156.01,159.44,...,37748300,-0.029111,2.941719,44.49,45.15,43.84,44.30,40910000,-0.427062,2.988139
1,"May 05, 2022",198.67,199.25,185.00,188.44,62633100,-5.149242,7.702703,163.85,164.08,...,43260400,-2.868250,4.377779,45.60,45.91,44.13,44.60,52996700,-2.192982,4.033537
2,"May 04, 2022",199.23,204.00,187.51,203.34,64885500,2.062942,8.794198,159.67,166.48,...,33599300,2.615096,5.113287,45.06,46.64,44.41,46.54,44625600,3.284510,5.021392
3,"May 03, 2022",194.00,198.25,191.33,196.02,47575100,1.041237,3.616788,158.15,160.71,...,25978600,-0.767714,1.420667,44.79,45.49,44.23,45.06,40158400,0.602813,2.848745
4,"May 02, 2022",185.41,195.74,183.91,195.33,57204900,5.350305,6.432494,156.71,158.23,...,35151100,2.434194,3.156904,43.72,44.99,43.51,44.96,45446500,2.836231,3.401517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,"Jun 07, 2012",3.14,3.16,2.97,2.97,52678000,-5.414013,6.397306,20.62,20.62,...,37792800,-1.383266,1.816935,26.37,26.43,25.89,25.94,29534200,-1.630641,2.085747
2496,"Jun 06, 2012",3.04,3.11,3.02,3.10,36896800,1.973684,2.980132,20.28,20.49,...,46860500,1.627424,1.943770,25.51,26.11,25.43,26.07,33553600,2.195218,2.674007
2497,"Jun 05, 2012",2.92,3.03,2.92,3.02,36522400,3.424658,3.767123,20.05,20.23,...,45715400,0.000000,1.268052,25.04,25.57,25.00,25.43,29491300,1.557508,2.280000
2498,"Jun 04, 2012",3.01,3.03,2.91,2.93,43285600,-2.657807,4.123711,20.05,20.27,...,47926300,-0.244584,1.624294,25.28,25.36,24.84,25.04,39903700,-0.949367,2.093398


In [56]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NVDA Date              2500 non-null   object 
 1   NVDA Open              2500 non-null   float64
 2   NVDA High              2500 non-null   float64
 3   NVDA Low               2500 non-null   float64
 4   NVDA Close             2500 non-null   float64
 5   NVDA Volume            2500 non-null   int64  
 6   NVDA Daily change      2500 non-null   float64
 7   NVDA Daily change Max  2500 non-null   float64
 8   AAPL Date              2500 non-null   object 
 9   AAPL Open              2500 non-null   float64
 10  AAPL High              2500 non-null   float64
 11  AAPL Low               2500 non-null   float64
 12  AAPL Close             2500 non-null   float64
 13  AAPL Volume            2500 non-null   int64  
 14  AAPL Daily change      2500 non-null   float64
 15  AAPL

In [69]:
tdf = pd.DataFrame(main_df, columns = ['NVDA Open','AAPL Open','AMD Open','MSFT Open','INTC Open','TSLA Open'])

In [71]:
tdf.tail()

,NVDA Open,AAPL Open,AMD Open,MSFT Open,INTC Open,TSLA Open
2495,3.14,20.62,6.10,29.64,26.37,5.96
2496,3.04,20.28,5.91,28.88,25.51,5.64
2497,2.92,20.05,5.66,28.51,25.04,5.57
2498,3.01,20.05,5.76,28.62,25.28,5.61
2499,3.04,20.33,5.90,28.76,25.40,5.71


In [80]:
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]

for index in indexes:
      
    main_df[index + " " +'Daily change'] = (((main_df[index + " " +'Close']).astype(float))-((main_df[index + " " +'Open']).astype(float)))/((main_df[index + " " +'Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((main_df[index + " " +'High'].astype(float)-main_df[index + " " +'Low'].astype(float))/main_df[index + " " +'Low'].astype(float)*100)


write_to_file(main_df, ("main_dataframe" + ".csv"))


In [81]:
main_df.tail()

,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
2495,"Jun 07, 2012",3.14,3.16,2.97,2.97,52678000,-5.414013,6.397306,20.62,20.62,...,37792800,-1.383266,1.816935,26.37,26.43,25.89,25.94,29534200,-1.630641,2.085747
2496,"Jun 06, 2012",3.04,3.11,3.02,3.10,36896800,1.973684,2.980132,20.28,20.49,...,46860500,1.627424,1.943770,25.51,26.11,25.43,26.07,33553600,2.195218,2.674007
2497,"Jun 05, 2012",2.92,3.03,2.92,3.02,36522400,3.424658,3.767123,20.05,20.23,...,45715400,0.000000,1.268052,25.04,25.57,25.00,25.43,29491300,1.557508,2.280000
2498,"Jun 04, 2012",3.01,3.03,2.91,2.93,43285600,-2.657807,4.123711,20.05,20.27,...,47926300,-0.244584,1.624294,25.28,25.36,24.84,25.04,39903700,-0.949367,2.093398
2499,"Jun 01, 2012",3.04,3.07,2.99,2.99,44098400,-1.644737,2.675585,20.33,20.45,...,56634300,-1.077886,1.828411,25.40,25.71,25.12,25.14,40246400,-1.023622,2.348726


,NVDA Open,AAPL Open,AMD Open,MSFT Open,INTC Open,TSLA Open
2495,3.14,20.62,6.10,29.64,26.37,5.96
2496,3.04,20.28,5.91,28.88,25.51,5.64
2497,2.92,20.05,5.66,28.51,25.04,5.57
2498,3.01,20.05,5.76,28.62,25.28,5.61
2499,3.04,20.33,5.90,28.76,25.40,5.71
